In [1]:
import random

In [2]:
# Arguments
REWARD = -0.01 # constant reward for non-terminal states
DISCOUNT = 0.9 # discount factor
MAX_ERROR = 10**(-3) # max error for convergence

In [3]:
# Set up the initial environment
NUM_ACTIONS = 4
ACTIONS = [(1, 0), (0, -1), (-1, 0), (0, 1)] # Down, Left, Up, Right
NUM_ROW = 3 # number of rows
NUM_COL = 4 # number of columns
U = [[0, 0, 0, 1], [0, 0, 0, -1], [0, 0, 0, 0], [0, 0, 0, 0]] # initial utility
policy = [[random.randint(0, 3) for j in range(NUM_COL)] for i in range(NUM_ROW)] # construct a random policy

In [4]:
# Visualization
def printEnvironment(arr, policy=False):
    """ Print the environment in a readable format """
    res = ""
    for r in range(NUM_ROW):
        res += "|"
        for c in range(NUM_COL):
            if r == c == 1:
                val = "WALL"
            elif r <= 1 and c == 3:
                val = "+1" if r == 0 else "-1"
            else:
                val = ["Down", "Left", "Up", "Right"][arr[r][c]]
            res += " " + val[:5].ljust(5) + " |" # format
        res += "\n"
    print(res)

In [5]:
# Get the utility of the state reached by performing the given action from the given state
def getU(U, r, c, action):
    """Return the utility of the state reached by performing the given action from the given state."""
    dr, dc = ACTIONS[action] # get the change in row and column
    newR, newC = r+dr, c+dc # get the new row and column
    if newR < 0 or newC < 0 or newR >= NUM_ROW or newC >= NUM_COL or (newR == newC == 1): # collide with the boundary or the wall
        return U[r][c]
    else:
        return U[newR][newC]

In [6]:
# Calculate the utility of a state given an action
def calculateU(U, r, c, action):
    """ Calculate the utility of a state given an action."""
    u = REWARD
    u += 0.1 * DISCOUNT * getU(U, r, c, (action-1)%4)
    u += 0.8 * DISCOUNT * getU(U, r, c, action)
    u += 0.1 * DISCOUNT * getU(U, r, c, (action+1)%4)
    return u

In [7]:
# Perform some simplified value iteration steps to get an approximation of the utilities
def policyEvaluation(policy, U):
    """ Perform some simplified value iteration steps to get an approximation of the utilities. """
    while True:
        nextU = [[0, 0, 0, 1], [0, 0, 0, -1], [0, 0, 0, 0], [0, 0, 0, 0]]
        error = 0
        for r in range(NUM_ROW):
            for c in range(NUM_COL):
                if (r <= 1 and c == 3) or (r == c == 1):
                    continue
                nextU[r][c] = calculateU(U, r, c, policy[r][c]) # simplified Bellman update
                error = max(error, abs(nextU[r][c]-U[r][c]))
        U = nextU
        if error < MAX_ERROR * (1-DISCOUNT) / DISCOUNT:
            break
    return U

In [8]:
def policyIteration(policy, U):
    """ Perform policy iteration. """
    print("During the policy iteration:\n")
    while True:
        U = policyEvaluation(policy, U)
        unchanged = True
        for r in range(NUM_ROW):
            for c in range(NUM_COL):
                if (r <= 1 and c == 3) or (r == c == 1):
                    continue
                maxAction, maxU = None, -float("inf")
                for action in range(NUM_ACTIONS):
                    u = calculateU(U, r, c, action)
                    if u > maxU:
                        maxAction, maxU = action, u
                if maxU > calculateU(U, r, c, policy[r][c]):
                    policy[r][c] = maxAction # the action that maximizes the utility
                    unchanged = False
        if unchanged:
            break
        printEnvironment(policy)
    return policy

In [9]:
# Print the initial environment
print("The initial random policy is:\n")
printEnvironment(policy)

# Policy iteration
policy = policyIteration(policy, U)

# Print the optimal policy
print("The optimal policy is:\n")
printEnvironment(policy)

The initial random policy is:

| Down  | Down  | Up    | +1    |
| Left  | WALL  | Right | -1    |
| Left  | Up    | Left  | Right |

During the policy iteration:

| Right | Right | Right | +1    |
| Up    | WALL  | Up    | -1    |
| Left  | Left  | Left  | Left  |

| Right | Right | Right | +1    |
| Up    | WALL  | Up    | -1    |
| Up    | Left  | Up    | Down  |

| Right | Right | Right | +1    |
| Up    | WALL  | Up    | -1    |
| Up    | Left  | Up    | Left  |

| Right | Right | Right | +1    |
| Up    | WALL  | Up    | -1    |
| Up    | Right | Up    | Left  |

The optimal policy is:

| Right | Right | Right | +1    |
| Up    | WALL  | Up    | -1    |
| Up    | Right | Up    | Left  |

